## Automatic

Since our target class is a bit complicated, we'll be joined in our implementation
journey by a simple class called `Greeter`.
The greeter is a simple class.
It takes a name on construction, and says "Hello".
On destruction, it says "Goodbye"

This will allow us to keep track of ctors and dtors as we progress through the talk.

In [5]:
class Greeter:
    def __init__(self, name):
        self.name = name
        print(f"Hello, {self.name}!")

    def close(self):
        print(f"Goodbye, {self.name}.")


def main():
    greeter = Greeter(1)
    print("We have a greeter!")
    greeter.close()

Our first implementation is as straight-forward as can be.
With a constructor and a "close" method to act like our dtor.

This is pretty straight-forward, as we already have context-managers and the `with` statement.
We just need to use them.

As we go through the implementation, we'll be using a simple class called `Greeter`.
`Greeter` is a friendly class to help us keep track of construction and destruction.

In [2]:
class Greeter:
    def __init__(self, name):
        self.name = name
        print(f"Hello, {self.name}!")

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Goodbye, {self.name}.")
        return False


def main():
    with Greeter(1):
        print("We have a greeter!")


main()

Hello, 1!
We have a greeter!
Goodbye, 1.


But what if we want more than one greeter?

Sure, we can use another `with` statement, but that doesn't work too well...

In [3]:
def main():
    with Greeter(1):
        print("First")
        with Greeter(2):
            print("Second")


main()

Hello, 1!
First
Hello, 2!
Second
Goodbye, 2.
Goodbye, 1.


It works, but you can see how the indentation would get you soon enough.

So, instead of using more and more `with` statement to stack
things up, let's use a proper stack!

In [1]:

class DtorScope:
    def __init__(self):
        self.stack = []

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        while self.stack:
            self.stack.pop().__exit__(exc_type, exc_val, exc_tb)

    def push(self, cm):
        self.stack.append(cm)

This is our own context manager.
It holds a stack of our objects, and when exiting the scope,
it calls their exit handlers in the right order.

In [4]:


def main():
    with DtorScope() as dtor_stack:
        greeter1 = Greeter(1)
        dtor_stack.push(greeter1)

        greeter2 = Greeter(2)
        dtor_stack.push(greeter2)


main()

Hello, 1!
Hello, 2!
Goodbye, 2.
Goodbye, 1.


This works.
It is automatic, it is fairly straight-forward, and it is way too explicit.

This is why our next part is...